# Assignment 2 - CIC-1205

## Exercise 1 - Cross Validation

Student: Balthazar Paixão


In [42]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pickle

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.tree import DecisionTreeClassifier, export_text

from sklearn.model_selection import GridSearchCV, StratifiedKFold


from sklearn.preprocessing import StandardScaler


In [24]:
SEED=57

def transform_array(arr: np.ndarray) -> list:
    """
    •0 → NONE
    •(0, 5] → WEAK
    •(5, 25] → MODERATE
    •(25,50] → STRONG
    •(50, ∞] → EXTREME
    """
    arr = arr.tolist()
    for i in range(len(arr)):
        if arr[i][0] == 0:
            arr[i] = 0  # 'NONE'
        elif arr[i][0] > 0 and arr[i][0] <= 5:
            arr[i] = 1  # 'WEAK'
        elif arr[i][0] > 5 and arr[i][0] <= 25:
            arr[i] = 2  # 'MODERATE'
        elif arr[i][0] > 25 and arr[i][0] <= 50:
            arr[i] = 3  # 'STRONG'
        elif arr[i][0] > 50:
            arr[i] = 4  # 'EXTREME'
    return arr


def count_values_list(lst: list) -> dict:
    """
    Count the number of times each value appears in a list
    """
    dict_counter = {}
    for i in lst:
        dict_counter[i] = dict_counter.get(i, 0) + 1

    return dict_counter

In [25]:
pickle_file = "../class-repo/cic1205/data/atmoseer/A652.pickle"

with open(pickle_file, "rb") as f:
    (X_train, y_train,
     X_val, y_val,
     X_test, y_test) = pickle.load(f)

In [26]:
y_train = transform_array(y_train)
print(f"y_train: {count_values_list(y_train)}")
y_val = transform_array(y_val)
print(f"y_val:   {count_values_list(y_val)}")
y_test = transform_array(y_test)
print(f"y_test:  {count_values_list(y_test)}")

y_train: {0: 9258, 2: 88, 1: 664, 3: 2}
y_val:   {0: 2277, 1: 222, 3: 1, 2: 6}
y_test:  {0: 8769, 1: 757, 2: 50, 4: 2, 3: 4}


In [27]:
X_train = np.concatenate((X_train, X_val), axis=0)
y_train = np.concatenate((y_train, y_val), axis=0)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


## K Means


In [36]:
params = {
    'n_clusters': [3, 4, 5],
    'max_iter': [100, 200],
    'tol': [0.0001, 0.001],
    'n_init': [10, 20]
}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
kmeans = KMeans(random_state=SEED)

grid_search = GridSearchCV(kmeans, params, scoring='f1_macro', cv=skf, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_score_)
print(grid_search.best_estimator_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


{'max_iter': 100, 'n_clusters': 3, 'n_init': 10, 'tol': 0.001}
0.17123434090670886
KMeans(max_iter=100, n_clusters=3, n_init=10, random_state=57, tol=0.001)
{'mean_fit_time': array([1.09553747, 1.02624145, 1.71574926, 1.51498747, 0.82245607,
       0.74269233, 1.95561423, 1.48763766, 1.69340372, 1.08437839,
       2.61126046, 2.27611628, 0.91700435, 0.71831274, 1.5082355 ,
       1.49625235, 0.80708699, 0.68791661, 1.91293573, 1.46695132,
       1.44244385, 1.25341992, 2.63422546, 2.08499184]), 'std_fit_time': array([0.331556  , 0.38126365, 0.39382094, 0.38418896, 0.14402158,
       0.22216614, 0.33595343, 0.33976703, 0.39061956, 0.19475749,
       0.40251431, 0.47401061, 0.16882853, 0.08544217, 0.31672129,
       0.10740276, 0.11711418, 0.13485818, 0.43148108, 0.48230036,
       0.09395007, 0.24330583, 0.36865185, 0.31373761]), 'mean_score_time': array([0.00882859, 0.01407714, 0.00794549, 0.00869274, 0.00901275,
       0.01097703, 0.01346316, 0.01291246, 0.01989737, 0.00967183,
      

## DB Scan


In [43]:
params = {
    'eps': [0.5, 1.0, 1.5],
    'min_samples': [5, 10, 15],
}
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

dbscan = DBSCAN()

grid_search = GridSearchCV(dbscan, params, cv=skf, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_score_)
print(grid_search.best_estimator_)


Fitting 5 folds for each of 9 candidates, totalling 45 fits


/home/balthapaixao/Desktop/baltha/projects/portfolio/MSc-codes/cic1205-machine-learning/.venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/balthapaixao/Desktop/baltha/projects/portfolio/MSc-codes/cic1205-machine-learning/.venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/balthapaixao/Desktop/baltha/projects/portfolio/MSc-codes/cic1205-machine-learning/.venv/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/home/balthapaixao/Desktop/baltha/projects/portfolio/MSc-codes/cic1205-machine-learning/.venv/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 353, in 

/home/balthapaixao/Desktop/baltha/projects/portfolio/MSc-codes/cic1205-machine-learning/.venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/balthapaixao/Desktop/baltha/projects/portfolio/MSc-codes/cic1205-machine-learning/.venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/balthapaixao/Desktop/baltha/projects/portfolio/MSc-codes/cic1205-machine-learning/.venv/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/home/balthapaixao/Desktop/baltha/projects/portfolio/MSc-codes/cic1205-machine-learning/.venv/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 353, in 

{'eps': 0.5, 'min_samples': 5}
nan
DBSCAN()


## Decision Tree

In [29]:
params = {
    'max_depth': [2, 3, 4, 5],
    'min_samples_split': [2, 3, 4, 5],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'max_features': [2, 3, 4, 5]
}

X_train = np.concatenate((X_train, X_val), axis=0)
y_train = np.concatenate((y_train, y_val), axis=0)

DT = DecisionTreeClassifier(random_state=SEED)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

grid_search = GridSearchCV(DT, params, scoring='f1_macro', cv=skf, verbose=1, n_jobs=-1)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

Fitting 5 folds for each of 320 candidates, totalling 1600 fits


{'max_depth': 5, 'max_features': 5, 'min_samples_leaf': 4, 'min_samples_split': 2}


In [30]:
DT = DecisionTreeClassifier(**grid_search.best_params_, random_state=SEED)

DT.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5, max_features=5, min_samples_leaf=4,
                       random_state=57)

In [31]:
text_representation = export_text(DT)
print(text_representation)

|--- feature_2 <= 0.94
|   |--- feature_10 <= 0.90
|   |   |--- feature_16 <= -0.33
|   |   |   |--- feature_15 <= -0.05
|   |   |   |   |--- feature_18 <= 1.09
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- feature_18 >  1.09
|   |   |   |   |   |--- class: 0
|   |   |   |--- feature_15 >  -0.05
|   |   |   |   |--- feature_7 <= 0.13
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- feature_7 >  0.13
|   |   |   |   |   |--- class: 1
|   |   |--- feature_16 >  -0.33
|   |   |   |--- feature_18 <= 0.90
|   |   |   |   |--- feature_18 <= -0.38
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- feature_18 >  -0.38
|   |   |   |   |   |--- class: 0
|   |   |   |--- feature_18 >  0.90
|   |   |   |   |--- feature_18 <= 0.92
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- feature_18 >  0.92
|   |   |   |   |   |--- class: 0
|   |--- feature_10 >  0.90
|   |   |--- feature_10 <= 1.06
|   |   |   |--- feature_15 <= 0.00
|   |   |   |   |--- feature_14 <= -1.39
|   |   